In [1]:
import zmq

class MTraderAPI:
    def __init__(self, host=None):
        self.HOST = host or '192.168.100.110'
        self.SYS_PORT = 15555       # REP/REQ port
        self.DATA_PORT = 15556      # PUSH/PULL port
        self.LIVE_PORT = 15557      # PUSH/PULL port
        self.EVENTS_PORT = 15558    # PUSH/PULL port

        # ZeroMQ timeout in seconds
        sys_timeout = 1
        data_timeout = 10

        # initialise ZMQ context
        context = zmq.Context()

        # connect to server sockets
        try:
            self.sys_socket = context.socket(zmq.REQ)
            # set port timeout
            self.sys_socket.RCVTIMEO = sys_timeout * 1000
            self.sys_socket.connect('tcp://{}:{}'.format(self.HOST, self.SYS_PORT))

            self.data_socket = context.socket(zmq.PULL)
            # set port timeout
            self.data_socket.RCVTIMEO = data_timeout * 1000
            self.data_socket.connect('tcp://{}:{}'.format(self.HOST, self.DATA_PORT))
        except zmq.ZMQError:
            raise zmq.ZMQBindError("Binding ports ERROR")

    def _send_request(self, data: dict) -> None:
        """Send request to server via ZeroMQ System socket"""
        try:
            self.sys_socket.send_json(data)
            msg = self.sys_socket.recv_string()
            # terminal received the request
            assert msg == 'OK', 'Something wrong on server side'
        except AssertionError as err:
            raise zmq.NotDone(err)
        except zmq.ZMQError:
            raise zmq.NotDone("Sending request ERROR")

    def _pull_reply(self):
        """Get reply from server via Data socket with timeout"""
        try:
            msg = self.data_socket.recv_json()
        except zmq.ZMQError:
            raise zmq.NotDone('Data socket timeout ERROR')
        return msg

    def live_socket(self, context=None):
        """Connect to socket in a ZMQ context"""
        try:
            context = context or zmq.Context.instance()
            socket = context.socket(zmq.PULL)
            socket.connect('tcp://{}:{}'.format(self.HOST, self.LIVE_PORT))
        except zmq.ZMQError:
            raise zmq.ZMQBindError("Live port connection ERROR")
        return socket

    def streaming_socket(self, context=None):
        """Connect to socket in a ZMQ context"""
        try:
            context = context or zmq.Context.instance()
            socket = context.socket(zmq.PULL)
            socket.connect('tcp://{}:{}'.format(self.HOST, self.EVENTS_PORT))
        except zmq.ZMQError:
            raise zmq.ZMQBindError("Data port connection ERROR")
        return socket

    def construct_and_send(self, **kwargs) -> dict:
        """Construct a request dictionary from default and send it to server"""

        # default dictionary
        request = {
            "action": None,
            "actionType": None,
            "symbol": None,
            "chartTF": None,
            "fromDate": None,
            "toDate": None,
            "id": None,
            "magic": None,
            "volume": None,
            "price": None,
            "stoploss": None,
            "takeprofit": None,
            "expiration": None,
            "deviation": None,
            "comment": None
        }

        # update dict values if exist
        for key, value in kwargs.items():
            if key in request:
                request[key] = value
            else:
                raise KeyError('Unknown key in **kwargs ERROR')

        # send dict to server
        self._send_request(request)

        # return server reply
        return self._pull_reply()
    
api = MTraderAPI()

In [2]:
rep = api.construct_and_send(action="ACCOUNT")
print(rep)

{'error': False, 'broker': 'MetaQuotes Software Corp.', 'currency': 'USD', 'server': 'MetaQuotes-Demo', 'trading_allowed': 1, 'bot_trading': 1, 'balance': 9244.8, 'equity': 9244.8, 'margin': 0.0, 'margin_free': 9244.8, 'margin_level': 0.0}


In [7]:
import time
import datetime
s = "12/3/2020"
time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())

1583946000.0

In [16]:
import datetime
timestamp = 1583947800
dt_object = datetime.datetime.fromtimestamp(timestamp)

print("dt_object =", dt_object)
#print("type(dt_object) =", type(dt_object))

dt_object = 2020-03-12 00:30:00


In [3]:
rep = api.construct_and_send(action="HISTORY", actionType="DATA", symbol="EURUSD", chartTF="H1", fromDate=1583946000.0)
print(rep)

{'data': [[1583946000, 1.13068, 1.13089, 1.12572, 1.128, 8181.0], [1583949600, 1.12797, 1.12807, 1.12603, 1.12686, 7175.0], [1583953200, 1.12685, 1.12884, 1.12666, 1.12845, 7056.0], [1583956800, 1.12845, 1.12878, 1.12717, 1.12816, 6412.0], [1583960400, 1.12815, 1.12825, 1.12685, 1.12725, 8868.0], [1583964000, 1.12725, 1.12796, 1.12682, 1.12682, 2357.0], [1583967600, 1.12682, 1.12689, 1.125, 1.12607, 906.0], [1583971200, 1.12602, 1.12663, 1.12543, 1.12585, 2272.0], [1583974800, 1.12585, 1.12629, 1.12541, 1.12613, 2925.0], [1583978400, 1.12613, 1.1276, 1.12594, 1.12714, 3153.0], [1583982000, 1.12714, 1.13332, 1.12662, 1.13189, 8852.0], [1583985600, 1.13189, 1.13314, 1.13055, 1.13209, 7500.0], [1583989200, 1.13207, 1.13209, 1.13075, 1.1313, 4166.0], [1583992800, 1.1313, 1.13135, 1.12854, 1.12959, 4996.0], [1583996400, 1.12957, 1.1299, 1.12911, 1.1293, 2653.0], [1584000000, 1.1293, 1.12975, 1.12854, 1.12925, 5093.0], [1584003600, 1.12926, 1.13195, 1.12889, 1.13119, 8360.0], [1584007200, 1.

In [4]:
import datetime
datetime.datetime.now()
datetime.datetime(2020,3,12)

datetime.datetime(2020, 3, 12, 0, 0)